# SPECFEM Users Workshop -- Day 1 (Oct. 5, 2022)

## Part 1B: Homogeneous Halfspace Model

Following up on Part 1A, this notebook allows Users to play around with their own SPECFEM2D homogeneous halfspace example. This is meant to familiarize Users with setting `SOURCE` and `STATION` attributes, adjust velocity model parameters, and assess their simulation work.

-----------

### 0) Relevant Information

>__NOTE:__ These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- !!! ADD DAY 1 SLIDES HERE !!!
- Workshop Material: https://github.com/adjtomo/adjdocs/tree/main/workshops/2022-10-05_specfem_users
- Today's Notebook: !!! ADD THIS !!!

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the `Run` button at the top, or by hitting `Shift + Enter`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. Where the number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands


## 1) Setting Up 

It is often desireable to run SPECFEM outside of the cloned repository, in order to keep files and outputs manageable. The trick here is that SPECFEM only requires 3 compenents for a sucessful simulation, the `bin/`, `DATA/`, and `OUTPUT_FILES/` directories. In this section we will set up a SPECFEM2D working directory that we can play around with.

>__NOTE:__ We will be doing all our work in the directory /home/scoped/work_day_1. All the following cells assume that we are in this directory, so you must evaluate the '%cd' command to ensure that cells work as expected.

In [ ]:
! mkdir /home/scoped/work_day_1
%cd /home/scoped/work_day_1

In [ ]:
# Symlink the binary files, and copy the relevant DATA/ directory
! ln -s /home/scoped/specfem2d/bin .
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
! mkdir OUTPUT_FILES

In [ ]:
! ls

### 2) Tape et al. 2007 Example

We will be working with example data from the [Tape et al. 2007 GJI publication](https://academic.oup.com/gji/article/168/3/1105/929373). This example contains two synthetic models, a number of seismic sources, and a list of 132 station locations. The two synthetic models are 1) a homogeneous halfspace model, and 2) a checkerboard model with approximately $\pm$10\% perturbations with respect to the homogeneous halfspace model of (1). 

In [ ]:
! ls DATA/

### a) The Homogeneous Halfspace Model

The homogeneous halfspace model in this example is defined in the `Par_file`, on Line 255. We can use the `seisflows sempar velocity_model` command to look at its values.

In [ ]:
! seisflows sempar -P DATA/Par_file_Tape2007_onerec velocity_model

According to the `Par_file` comments, the model parameter values represent the following:
`model_number 1 rho Vp Vs 0 0 QKappa Qmu  0 0 0 0 0 0`. 

Therefore, the homogeneous halfspace model defines a region with P-wave velocity 5.8km/s and S-wave velocity 3.5km/s.

### b) Visualizing the Checkerboard

We can use Matplotlib and NumPy to help us visualize these a bit better. First we'll have a look at the checkerboard model data file, and then we'll plot it directly.

In [ ]:
# The columns of the data file correspond to the following:
# line_no x[m] z[m] density vp[m/s] vs[m/s]
! head DATA/model_velocity.dat_checker

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Only grabbing X, Z, Vs and Vp
data = np.genfromtxt("DATA/model_velocity.dat_checker", dtype=float, usecols=[1,2,4,5])
chkbd_x, chkbd_z, chkbd_vp, chkbd_vs = data.T

In [ ]:
# Plotting Vp
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vp, levels=125, cmap="seismic_r")
plt.xlabel("X [m]")
plt.ylabel("Z [m]")
plt.title("Checkerboard Vp [m/s]")
plt.colorbar()

In [ ]:
# Plotting Vs
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vs, levels=125, cmap="seismic_r")
plt.xlabel("X [m]")
plt.ylabel("Z [m]")
plt.title("Checkerboard Vs [m/s]")
plt.colorbar()

### c) Visualizing Source-Receiver Geometry

We can similarly plot the SOURCES and STATIONS available to see what the experiemental setup looks like.
We can use Python to grab Cartesian coordinate values from these files.

In [ ]:
# Grab STATIONS data
! head DATA/STATIONS_checker

# Small code snippet to grab coordinates from STATIONS file
sta_x, sta_z = np.genfromtxt("DATA/STATIONS_checker", dtype=float, usecols=[2, 3]).T
print(sta_x[:10])

In [ ]:
# There are 25 sources
# `xs` is defined on line 2, `zs` on line 3
! ls DATA/SOURCE_???
! echo
! head DATA/SOURCE_001

In [ ]:
# Small code snippet to grab coordinates from SOURCE files
ev_x, ev_z = [], []
for i in range(1, 26):
    source_file = f"DATA/SOURCE_{i:0>3}"
    with open(source_file, "r") as f:
        lines = f.readlines()
    # Trying to break apart the following line
    # 'xs = 299367.72      # source location x in meters\n'
    xs = float(lines[2].split("=")[1].split("#")[0].strip())
    zs = float(lines[3].split("=")[1].split("#")[0].strip())
    
    ev_x.append(xs)
    ev_z.append(zs)

print(ev_x[:10])

In [ ]:
# Plot SOURCES and STATIONS together. Annotate names
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vs, levels=250, cmap="seismic_r")
plt.scatter(ev_x, ev_z, c="y", marker="*", s=100, edgecolor="k")
plt.scatter(sta_x, sta_z, c="c", marker="v", s=20, edgecolor="k")
plt.title("SOURCE-RECEIVER GEOMETRY")

In [ ]:
# Plot SOURCES next to source names
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vs, levels=250, cmap="seismic_r", alpha=0.1)
for i, (x, z) in enumerate(zip(ev_x, ev_z)):
    plt.scatter(x, z, c="y", marker="*", s=45)
    plt.text(x, z, f"{i:0>3}")
plt.title("SOURCES")

In [ ]:
# Plot SOURCES and STATIONS together. Annotate names
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vs, levels=250, cmap="seismic_r", alpha=0.1)
for i, (x, z) in enumerate(zip(sta_x, sta_z)):
    plt.scatter(x, z, c="c", marker="v", s=8)
    plt.text(x, z, f"{i:0>3}", fontsize=9)
plt.title("STATIONS")

In the above figures, the upside-down blue triangles represent the 132 receivers in this example, while the 25 yellow stars are the sources. Now that we are familiar with our experimental setup, we can run SPECFEM2D to generate synthetics.

## 2) Running the Example

To run the example, we'll have to do a some setup of our working directory to get files in the correct place. 

In [ ]:
! ls DATA

### a) Choose a Source file

SPECFEM2D will look for a file named `SOURCE` in the *DATA/* directory. There are 25 sources to choose from. You can have a look at the SOURCE plot we created in the previous section to choose which SOURCE
you'd like to run.

In [ ]:
# CHOOSE your source file here by replacing 'SOURCE_001'
! cp -f DATA/SOURCE_001 DATA/SOURCE

In [ ]:
! head -1 DATA/SOURCE

### b) Choosing Stations